In [1]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel

### Import Dependencies and Libraries

This section imports all necessary libraries for our multi-model agent system:

#### Core Agent Framework:
- **`agents`**: Main framework components
  - `Agent`: Creates AI agents with custom behaviors
  - `Runner`: Executes agents asynchronously
  - `trace`: Provides execution monitoring
  - `function_tool`: Decorator for creating agent tools
  - `OpenAIChatCompletionsModel`: Custom model wrapper
  - `input_guardrail`: Security validation decorator
  - `GuardrailFunctionOutput`: Guardrail response structure

#### External Services:
- **`sendgrid`**: Email delivery service
- **`AsyncOpenAI`**: Async OpenAI client for multiple providers
- **`pydantic`**: Data validation and serialization
- **`dotenv`**: Environment variable management

#### Type Safety:
- **`typing.Dict`**: Type hints for better code quality

# Advanced Multi-Model Agent System with Guardrails

## 📋 Project Overview
This advanced notebook demonstrates the implementation of a sophisticated multi-agent email system with safety guardrails. The system combines multiple AI models (OpenAI, Google Gemini, Meta Llama) to create diverse sales emails while implementing security controls to prevent misuse.

## 🎯 Learning Objectives
By completing this notebook, you will:
- **Multi-Model Integration**: Work with OpenAI, Gemini, and Llama models simultaneously
- **Agent Orchestration**: Coordinate multiple specialized agents for complex workflows
- **Guardrail Implementation**: Build security controls to prevent inappropriate agent usage
- **Production Patterns**: Implement enterprise-grade safety and monitoring systems
- **Email Automation**: Create a complete email generation and delivery pipeline

## 🏗️ System Architecture
```
Sales Manager Agent
├── Gemini Sales Agent (Humorous Style)
├── Llama Sales Agent (Concise Style)
├── Email Manager Agent
│   ├── Subject Writer
│   ├── HTML Converter
│   └── SendGrid Integration
└── Input Guardrails
    └── Name Detection System
```

## 🔑 Key Features
- **Multi-model comparison** for optimal email generation
- **Input validation** with custom guardrails
- **Professional email formatting** with HTML conversion
- **Secure API key management** across multiple providers
- **Error handling** and safety mechanisms

---

## Part 1: Environment Setup and Multi-Provider Configuration

In [2]:
load_dotenv(override=True)

True

### Environment Variable Configuration

Load environment variables with override to ensure latest API keys are used. This is critical for multi-provider authentication.

In [3]:
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")
if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AI
Groq API Key exists and begins gsk_


### Multi-Provider API Key Validation

Comprehensive validation of API keys across multiple AI providers:

#### Security Best Practices:
- **Partial key display**: Shows first few characters for verification without exposing full keys
- **Optional provider handling**: Graceful handling of missing optional API keys
- **Validation feedback**: Clear status messages for each provider

#### Provider Information:
- **OpenAI**: Primary LLM provider (Required)
- **Google API**: For Gemini model access (Optional but recommended)
- **Groq API**: For Llama model access (Optional but recommended)

This validation ensures proper authentication before proceeding with agent creation.

In [4]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

## Part 2: Agent Personality Definition

### Sales Agent Instructions Configuration

Three distinct sales approaches for ComplAI's SOC2 compliance tool:

#### Agent 1: Professional & Serious (Not implemented in this version)
- **Target**: Enterprise decision-makers
- **Style**: Formal, authoritative, compliance-focused
- **Tone**: Professional and trustworthy

#### Agent 2: Humorous & Engaging (Gemini)
- **Target**: Tech-savvy professionals
- **Style**: Witty, engaging, personable
- **Purpose**: Break through inbox clutter with humor
- **Goal**: Higher response rates through personality

#### Agent 3: Concise & Direct (Llama)
- **Target**: Busy executives
- **Style**: Straight to the point, efficient
- **Purpose**: Respect time constraints
- **Goal**: Quick decision-making with clear value props

### ComplAI Company Context:
- **Product**: SaaS tool for SOC2 compliance
- **Technology**: AI-powered audit preparation
- **Market**: B2B compliance and security sector

In [5]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"

### External Provider Base URLs

Configuration for accessing non-OpenAI models through OpenAI-compatible APIs:

#### Google Gemini Integration:
- **Base URL**: Google's generative AI endpoint with OpenAI compatibility
- **Purpose**: Access to Gemini 2.0 Flash model
- **Benefits**: Google's latest multimodal capabilities

#### Groq Integration:
- **Base URL**: Groq's high-speed inference endpoint
- **Purpose**: Access to Meta's Llama 3.3 70B model
- **Benefits**: Ultra-fast inference for production workloads

In [6]:

gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)

gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
llama3_3_model = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)

## Part 3: Multi-Model Client Configuration

### Asynchronous Client Setup

Creating specialized clients for each AI provider:

#### Client Configuration:
```python
# Gemini Client (Google)
gemini_client = AsyncOpenAI(
    base_url=GEMINI_BASE_URL,
    api_key=google_api_key
)

# Groq Client (Meta Llama)
groq_client = AsyncOpenAI(
    base_url=GROQ_BASE_URL, 
    api_key=groq_api_key
)
```

#### Model Wrapper Creation:
- **`OpenAIChatCompletionsModel`**: Standardizes different providers under one interface
- **Benefits**: Uniform API regardless of underlying model provider
- **Flexibility**: Easy switching between models without code changes

#### Model Specifications:
- **Gemini 2.0 Flash**: Google's latest multimodal model with fast response times
- **Llama 3.3 70B Versatile**: Meta's open-source model with broad capabilities

In [7]:
sales_agent2 =  Agent(name="Gemini Sales Agent", instructions=instructions2, model=gemini_model)
sales_agent3  = Agent(name="Llama3.3 Sales Agent",instructions=instructions3,model=llama3_3_model)

### Agent Instantiation with Custom Models

Creating specialized sales agents using different AI models:

#### Agent Configuration:
- **Name**: Unique identifier for tracking and debugging
- **Instructions**: Personality and behavior definition
- **Model**: Specific AI model for each agent's strengths

#### Agent Specializations:
1. **Gemini Sales Agent**: 
   - Uses Google's Gemini 2.0 Flash
   - Humorous and engaging personality
   - Optimized for creative, witty content

2. **Llama Sales Agent**:
   - Uses Meta's Llama 3.3 70B via Groq
   - Concise and direct approach
   - Optimized for efficient, clear communication

This multi-model approach leverages each model's unique strengths for optimal results.

In [8]:
description = "Write a cold sales email"

tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

## Part 4: Agent-to-Tool Conversion

### Converting Agents into Reusable Tools

Transform agents into tools that can be used by higher-level orchestrator agents:

#### Tool Conversion Process:
```python
tool2 = sales_agent2.as_tool(
    tool_name="sales_agent2",
    tool_description="Write a cold sales email"
)
```

#### Benefits of Agent-as-Tool Pattern:
- **Modularity**: Agents become reusable components
- **Orchestration**: Higher-level agents can coordinate multiple specialists
- **Abstraction**: Complex agent logic hidden behind simple tool interface
- **Scalability**: Easy to add new agents without changing orchestrator code

#### Tool Properties:
- **tool_name**: Unique identifier for the orchestrator to reference
- **tool_description**: Helps the orchestrator understand when to use this tool

In [9]:

@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("sonudevmail16@gmail.com")
    to_email = To("abhinavsarkar53@gmail.com") # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [10]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [11]:
email_tools = [subject_tool, html_tool, send_html_email]

In [12]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

In [13]:
tools = [tool2, tool3]
handoffs = [emailer_agent]

In [14]:
sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

In [15]:
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

guardrail_agent = Agent( 
    name="Name check",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput,
    model="gpt-4o-mini"
)

In [16]:
@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output},tripwire_triggered=is_name_in_message)

In [17]:
careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_name]
    )

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

In [18]:
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

guardrail_agent = Agent( 
    name="Name check",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput,
    model="gpt-4o-mini"
)

In [19]:
@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output},tripwire_triggered=is_name_in_message)

In [20]:
careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_name]
    )

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

In [21]:

message = "Send out a cold sales email addressed to Dear CEO from Head of Business Development"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)